In [35]:
import pygame
import random
import math
import time
CAR_MOVEMENT_DELAY = 1000  # Delay in milliseconds
CAR_CHECK_DELAY = 1  # Adjust the delay time as needed (in seconds)
delivery_points = []

In [36]:
def draw_grid_with_entities(screen, width, height, rows, cols, house_image, house_locations, robot_image, robot_location, ambulance_image, ambulance_location, car1_image, car1_image_locations, car2_image, car2_image_locations):
    """
    Draw a grid on the screen with house images, robot image, ambulance image, car1 images, and car2 images at the specified locations.

    Parameters:
        screen (pygame.Surface): The surface to draw on.
        width (int): Width of the screen.
        height (int): Height of the screen.
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        house_image (pygame.Surface): The image of the house.
        house_locations (list): List of locations for the house [(row, col), ...].
        robot_image (pygame.Surface): The image of the robot.
        robot_location (tuple): The location of the robot (row, col).
        ambulance_image (pygame.Surface): The image of the ambulance.
        ambulance_location (tuple): The location of the ambulance (row, col).
        car1_image (pygame.Surface): The image of car1.
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image (pygame.Surface): The image of car2.
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
    """
    cell_width = width // cols
    cell_height = height // rows

    screen.fill((135 , 206 , 235))

    for x in range(0, width + 1, cell_width):  # Increment the range by 1 to ensure the rightmost line is drawn
        pygame.draw.line(screen, (255, 255, 255), (x, 0), (x, height))  # Set border color to white
    for y in range(0, height + 1, cell_height):  # Increment the range by 1 to ensure the bottom line is drawn
        pygame.draw.line(screen, (255, 255, 255), (0, y), (width, y))  # Set border color to white

    # Adjust cell size to increase the size of the cell in the grid
    cell_width += 2
    cell_height += 2

    # Draw house
    for location in house_locations:
        house_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        house_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        house_image_resized = pygame.transform.scale(house_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(house_image_resized, (house_x, house_y))

    # Draw robot
    robot_x = robot_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    robot_y = robot_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    robot_image_resized = pygame.transform.scale(robot_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
    screen.blit(robot_image_resized, (robot_x, robot_y))

    # Draw ambulance
    ambulance_x = ambulance_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    ambulance_y = ambulance_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    ambulance_image_resized = pygame.transform.scale(ambulance_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness

    screen.blit(ambulance_image_resized, (ambulance_x, ambulance_y))

    # Draw car1 images
    for location in car1_image_locations:
        car1_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car1_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car1_image_resized = pygame.transform.scale(car1_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car1_image_resized, (car1_x, car1_y))

    # Draw car2 images
    for location in car2_image_locations:
        car2_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car2_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car2_image_resized = pygame.transform.scale(car2_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car2_image_resized, (car2_x, car2_y))


In [37]:
def generate_location(rows, cols, excluded_locations):
    """
    Generate a random location that does not overlap with the excluded locations.

    Parameters:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        excluded_locations (list): List of locations to avoid [(row, col), ...].

    Returns:
        tuple: The random location (row, col).
    """
    while True:
        location = (random.randint(0, rows - 1), random.randint(0, cols - 1))
        if location not in excluded_locations and location != (0, 0):  # Exclude the (0, 0) location
            return location


In [38]:
def move_ambulance(ambulance_location, rows, cols, house_locations, robot_location, car1_image_locations, car2_image_locations, prev_direction=None):
    """
    Move the ambulance randomly within the grid, ensuring it does not collide with house, robot, or other cars.

    Parameters:
        ambulance_location (tuple): The current location of the ambulance (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        house_locations (list): List of locations for the house [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of the ambulance(dx, dy).

    Returns:
        tuple: The new location of the ambulance (row, col).
        tuple: The new movement direction of the ambulance (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby cars or robot
    nearby_entities = []
    for dx, dy in movements:
        new_row = ambulance_location[0] + dx
        new_col = ambulance_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) in car1_image_locations or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = ambulance_location[0] + dx
        new_col = ambulance_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with house
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in house_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = ambulance_location[0] + dx
        new_col = ambulance_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with house
        if not (0 <= new_row < rows and 0 <= new_col < cols) or (new_row, new_col) in house_locations:
            continue

        # If the new location is valid, return it along with the movement direction
        return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return ambulance_location, (0, 0)


def move_car1(car1_location, rows, cols, house_locations, robot_location, ambulance_location, car2_image_locations, prev_direction=None):
    """
    Move car1 randomly within the grid, ensuring it does not collide with house, robot, ambulance, or other cars.

    Parameters:
        car1_location (tuple): The current location of car1 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        house_locations (list): List of locations for the house [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        ambulance_location (tuple): The location of the ambulance (row, col).
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car1 (dx, dy).

    Returns:
        tuple: The new location of car1 (row, col).
        tuple: The new movement direction of car1 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == ambulance_location or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with house
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in house_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in house_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != ambulance_location) and \
           ((new_row, new_col) not in car2_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car1_location, (0, 0)


def move_car2(car2_location, rows, cols, house_locations, robot_location, ambulance_location, car1_image_locations, prev_direction=None):
    """
    Move car2 randomly within the grid, ensuring it does not collide with house, robot, ambulance, or other cars.

    Parameters:
        car2_location (tuple): The current location of car2 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        house_locations (list): List of locations for the house [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        ambulance_location (tuple): The location of the ambulance (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car2 (dx, dy).

    Returns:
        tuple: The new location of car2 (row, col).
        tuple: The new movement direction of car2 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided and it's in the list of movements, prioritize that direction
    if prev_direction and prev_direction in movements:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == ambulance_location or \
           (new_row, new_col) in car1_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with house
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in house_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in house_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != ambulance_location) and \
           ((new_row, new_col) not in car1_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car2_location, (0, 0)




In [39]:
def draw_item_buttons(screen, item_images):
    # Fixed size for the item buttons
    item_button_size = 50
    
    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300

    # Draw item buttons below the grid
    for i, item_image in enumerate(item_images):
        # Calculate the position of the item button
        button_rect = pygame.Rect(start_x + i * item_button_size, screen.get_height() - item_button_size, item_button_size, item_button_size)
        
        pygame.draw.rect(screen, (0, 255, 0), button_rect, 1)  # Draw button border

        # Resize the item image to fit the button size (square dimensions)
        resized_item_image = pygame.transform.scale(item_image, (item_button_size - 2, item_button_size - 2))  # Subtract 2 for border thickness

        # Calculate the position to center the image within the button
        image_x = start_x + i * item_button_size + (item_button_size - resized_item_image.get_width()) // 2
        image_y = screen.get_height() - item_button_size + (item_button_size - resized_item_image.get_height()) // 2

        screen.blit(resized_item_image, (image_x, image_y))  # Draw item image




def handle_item_click(index, item_images, screen, rows, cols):

    # Remove the clicked item image from the list
    del item_images[index]

    # Calculate the area of the screen to redraw (where the item buttons are located)
    cell_height = screen.get_height() // (rows + 1)

    # Calculate the width of each column (grid cells)
    cell_width = screen.get_width() // cols

    # Calculate the width and height of each item button (square dimensions)
    item_button_size=50

    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300

    # Redraw only the button area to reflect the changes
    screen.fill((255, 255, 255), (0, screen.get_height() - item_button_size, screen.get_width(), item_button_size))  # Fill only the button area with white color
    draw_item_buttons(screen, item_images)  # Draw item buttons
    pygame.display.update((0, screen.get_height() - item_button_size, screen.get_width(), item_button_size))  # Update only the button area of the display




In [40]:
def euclidean_distance(start, goal):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((start[0] - goal[0]) ** 2 + (start[1] - goal[1]) ** 2)

def heuristic_with_obstacles(start, goal, obstacles):
    """
    Calculate the heuristic function with obstacles using Euclidean distance.
    Penalize the heuristic if there are obstacles between the start and the goal.
    """
    distance = euclidean_distance(start, goal)
    # Check for obstacles between start and goal
    for obstacle in obstacles:
        if obstacle[0] > min(start[0], goal[0]) and obstacle[0] < max(start[0], goal[0]) and obstacle[1] > min(start[1], goal[1]) and obstacle[1] < max(start[1], goal[1]):
            # Penalize the heuristic if there is an obstacle between start and goal
            distance += 100  # Adjust the penalty as needed
            break
    return distance

def best_first_search(start, goal, house_locations):
    """
    Perform Best-First Search algorithm to find the optimal path from start to goal.
    """
    space_complexity=0
    time_complexity=0
    visited = set()  # Set to store visited nodes
    queue = [(heuristic_with_obstacles(start, goal, house_locations), [start])]  # Priority queue for nodes to be explored

    while queue:
        # Get the node with the lowest heuristic value
        queue.sort()  # Sort the queue based on heuristic value
        _, path = queue.pop(0)
        node = path[-1]

        # Check if the goal is reached
        if node == goal:
            return path , time_complexity , space_complexity

        # Explore neighboring nodes
        for neighbor in get_neighbors_within_grid(node, house_locations):
            if neighbor not in visited:
                visited.add(neighbor)
                time_complexity=time_complexity+1
                new_path = list(path)
                new_path.append(neighbor)
                queue.append((heuristic_with_obstacles(neighbor, goal, house_locations), new_path))
                if (space_complexity<(len(visited))+(len(queue))):
                    space_complexity=len(visited)+len(queue)

    return None ,time_complexity , space_complexity # No path found

def get_neighbors_within_grid(node, house_locations):
    """
    Get neighboring nodes of a given node within the 15x15 grid excluding house locations.
    """
    neighbors = []
    x, y = node
    # Define possible movements (up, down, left, right)
    movements = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    # Add neighbors that are within the grid and not in house locations
    for dx, dy in movements:
        new_x, new_y = x + dx, y + dy
        if (0 <= new_x < 15) and (0 <= new_y < 15) and ((new_x, new_y) not in house_locations):
            neighbors.append((new_x, new_y))
    return neighbors


In [41]:
def generate_delivery_points(rows, cols, excluded_locations):
    """
    Generate 5 random delivery points that do not overlap with excluded locations.

    Parameters:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        excluded_locations (list): List of locations to avoid [(row, col), ...].

    Returns:
        list: List of 5 random delivery points [(row, col), ...].
    """
    global delivery_points
    if len(delivery_points) < 5:
        while len(delivery_points) < 5:
            location = generate_location(rows, cols, excluded_locations)
            # Check if the location is not in the immediate vicinity of the first 10 excluded locations
            if all(abs(location[0] - ex_loc[0]) > 1 or abs(location[1] - ex_loc[1]) > 1 for ex_loc in excluded_locations[:10]):
                delivery_points.append(location)
                excluded_locations.append(location)
                if len(delivery_points) == 5:
                    break

    return delivery_points[:5]  # Return only the first 5 delivery points


def delivery_buttons(delivery_points, cell_width, cell_height):
    """
    Create buttons for the cells where delivery points are located.

    Parameters:
        delivery_points (list): List of delivery points [(row, col), ...].
        cell_width (int): Width of each cell in the grid.
        cell_height (int): Height of each cell in the grid.

    Returns:
        list: List of button rects for each delivery point.
    """
    button_rects = []
    for delivery_point in delivery_points:
        delivery_x = delivery_point[1] * cell_width
        delivery_y = delivery_point[0] * cell_height
        button_rect = pygame.Rect(delivery_x, delivery_y, cell_width, cell_height)
        button_rects.append(button_rect)
    return button_rects

def display_coordinates(x, y, screen, cell_width, cell_height, point_image_resized, delivery_points):
    # print("Clicked on delivery point at (x={}, y={})".format(x, y))
    for i, point in enumerate(delivery_points):
        point_x = point[1] * cell_width
        point_y = point[0] * cell_height
        if point_x == x and point_y == y:
            pygame.draw.rect(screen, (255, 0, 0), (point_x, point_y, cell_width, cell_height))
            screen.blit(point_image_resized, (point_x, point_y))


In [42]:
def is_occupied(next_location ,  car1_image_locations, car2_image_locations , ambulance_location):
    x, y = next_location
    if (x, y) in car1_image_locations or (x, y) in car2_image_locations or (x,y) in ambulance_location:
        return True  # Location is occupied by a car
    return False



In [43]:
def delivery_or_not(prev,next):
    if (prev>next):
        return True
    else:
        return False
class statically:
    static_variable = 0  # This is a static variable

    def __init__(self, value):
        self.value = value

    def increment_static_variable(self):
        statically.static_variable += 1
        return statically.static_variable

In [44]:
def euclidean_distance(start, goal):
    """
    Calculate the Euclidean distance between two points.
    
    Args:
    - start: Tuple (x, y) representing the start point
    - goal: Tuple (x, y) representing the goal point
    
    Returns:
    - Euclidean distance between the start and goal points
    """
    return math.sqrt((start[0] - goal[0]) ** 2 + (start[1] - goal[1]) ** 2)

def generate_costs_15x15():
    """
    Generate random costs between 1 to 20 for each location on a 15 by 15 grid.
    
    Returns:
    - A 2D array representing the costs between each location and the goal.
    """
    costs = [[0] * 15 for _ in range(15)]  # Initialize 2D array with zeros
    
    for i in range(15):
        for j in range(15):
            # Generate random cost between 1 to 20
            costs[i][j] = random.randint(1, 20)
    
    return costs

def a_star(start, goal, obstacles, costs):
    """
    Perform A* algorithm to find the optimal path from start to goal.
    """
    space_complexity = 0
    time_complexity = 0
    visited = set()  # Set to store visited nodes
    queue = [(heuristic_with_obstacles(start, goal, obstacles), [start])]  # Priority queue for nodes to be explored

    while queue:
        # Get the node with the lowest f-value
        queue.sort()  # Sort the queue based on f-value
        _, path = queue.pop(0)
        node = path[-1]

        # Check if the goal is reached
        if node == goal:
            return path, time_complexity, space_complexity

        # Explore neighboring nodes
        for neighbor in get_neighbors_within_grid(node, obstacles):
            if neighbor not in visited:
                visited.add(neighbor)
                time_complexity += 1
                new_path = list(path)
                new_path.append(neighbor)
                g_cost = len(new_path) + costs[neighbor[0]][neighbor[1]]  # Cost from start to neighbor
                h_cost = heuristic_with_obstacles(neighbor, goal, obstacles)  # Heuristic cost from neighbor to goal
                f_cost = g_cost + h_cost
                queue.append((f_cost, new_path))
                if space_complexity < (len(visited) + len(queue)):
                    space_complexity = len(visited) + len(queue)

    return None, time_complexity, space_complexity  # No path found


In [45]:


def main():
    # Initialize Pygame
    pygame.init()
    obj1 = statically(0)
    check_robot=0
    prev_items=0
    curr_items=1000

    # Set up the screen
    screen_width, screen_height = 1000, 650  # Increase the screen height to accommodate the item buttons
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Grid with houses, Robot, ambulance, Cars")

    # Load house, robot, ambulance, car1, and car2 images
    # (Assuming these resources are loaded properly)
    house_image = pygame.image.load("house.png")
    robot_image = pygame.image.load("robot.jpg")
    ambulance_image = pygame.image.load("ambulance.png")
    car1_image = pygame.image.load("car1.jpg")
    car2_image = pygame.image.load("car2.jpg")

    # Load item images
    item1_image = pygame.image.load("item1.jpeg")
    item2_image = pygame.image.load("item2.jpeg")
    item3_image = pygame.image.load("item3.jpeg")
    item4_image = pygame.image.load("item4.png")
    item5_image = pygame.image.load("item5.png")
    item6_image = pygame.image.load("item6.jpeg")
    item7_image = pygame.image.load("item7.jpg")
    item8_image = pygame.image.load("item8.jpeg")

    # Create a list of item images
    item_images = [item1_image, item2_image, item3_image, item4_image, item5_image, item6_image, item7_image, item8_image]

    # Define grid size
    rows, cols = 15, 15

    # Generate random house locations
    house_locations = []
    for _ in range(10):
        house_locations.append(generate_location(rows, cols, house_locations))

    # Generate random locations for robot, ambulance, car1, and car2, ensuring they don't overlap with houses
    robot_location = (0, 0)
    ambulance_location = generate_location(rows, cols, house_locations + [robot_location])
    car1_image_locations = []
    for _ in range(2):
        car1_image_locations.append(generate_location(rows, cols, house_locations + [robot_location] + [ambulance_location] + car1_image_locations))
    car2_image_locations = []
    for _ in range(2):
        car2_image_locations.append(generate_location(rows, cols, house_locations + [robot_location] + [ambulance_location] + car1_image_locations + car2_image_locations))

    prev_direction_ambulance = None  # Initialize previous direction for ambulance as None
    prev_direction_car1 = [None, None]  # Initialize previous direction for car1s as None for both cars
    prev_direction_car2 = [None, None]  # Initialize previous direction for car2s as None for both cars

    # Calculate cell width and height
    cell_width = screen_width // cols
    cell_height = (screen_height - 50) // rows  # Subtract 50 for the height of the item buttons row

    item_button_size = 50

    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300
    button_rects = []
    for i, item_image in enumerate(item_images):
        button_rect = pygame.Rect(start_x + i * item_button_size, screen_height - item_button_size, item_button_size,
                                   item_button_size)
        button_rects.append(button_rect)
    item_name = ["item0", "item1", "item2", "item3", "item4", "item5", "item6", "item7"]
    value = 100

    # List to store clicked delivery points
    clicked_points = []

    del_x = None
    del_y = None
    click_on_delivery = None
    goal_location = None  # Initialize goal_location

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pass
            
                # Check if the mouse click was on an item button
            # Check if the mouse click was on an item button
                mouse_x, mouse_y = event.pos
                for i, button_rect in enumerate(delivery_buttons_rects):
                    if button_rect.collidepoint(mouse_x, mouse_y):
                        # print("prev:",prev_items,"Curr:",curr_items)
                        if click_on_delivery is None and delivery_or_not(prev_items,curr_items):
                            obj1.increment_static_variable()
                            click_on_delivery = 1
                            # Get the delivery point coordinates corresponding to the clicked button
                            del_x, del_y = delivery_points[i]
                            # print("Clicked Delivery Point Location:", del_x, del_y)
                            # Calculate the screen coordinates for displaying the clicked point
                            x = del_x * cell_width + cell_width // 2
                            y = del_y * cell_height + cell_height // 2
                            display_coordinates(x, y, screen, cell_width, cell_height, point_image_resized, delivery_points)
                            clicked_points.append((x, y))  # Add the clicked point to the list


                if del_x is not None and del_y is not None:
                    goal_location = (del_x, del_y)
                    # Clear the delivery point variables for the next delivery
                    del_x, del_y = None, None

                prev_items=len(item_name)

                if mouse_x >= 300 and mouse_x < 350 and mouse_y >= 600 and mouse_y < 650:
                    value = 0
                elif mouse_x >= 350 and mouse_x < 400 and mouse_y >= 600 and mouse_y < 650:
                    value = 1
                elif mouse_x >= 400 and mouse_x < 450 and mouse_y >= 600 and mouse_y < 650:
                    value = 2
                elif mouse_x >= 450 and mouse_x < 500 and mouse_y >= 600 and mouse_y < 650:
                    value = 3
                elif mouse_x >= 500 and mouse_x < 550 and mouse_y >= 600 and mouse_y < 650:
                    value = 4
                elif mouse_x >= 550 and mouse_x < 600 and mouse_y >= 600 and mouse_y < 650:
                    value = 5
                elif mouse_x >= 600 and mouse_x < 650 and mouse_y >= 600 and mouse_y < 650:
                    value = 6
                elif mouse_x >= 650 and mouse_x < 700 and mouse_y >= 600 and mouse_y < 650:
                    value = 7

                if len(item_name) > value:
                    handle_item_click(value, item_images, screen, rows, cols)
                    del item_name[value]
                    value = 100
                    curr_items=len(item_name)

        if robot_location == goal_location:
            
            # print("check",check_robot)    
            click_on_delivery = None
            

        # Move the ambulance
        ambulance_location, prev_direction_ambulance = move_ambulance(ambulance_location, rows, cols, house_locations,
                                                              robot_location, car1_image_locations,
                                                              car2_image_locations, prev_direction_ambulance)

        # Move car1s
        for i, car1_location in enumerate(car1_image_locations):
            car1_image_locations[i], prev_direction_car1[i] = move_car1(car1_location, rows, cols,
                                                                        house_locations, robot_location,
                                                                        ambulance_location, car2_image_locations,
                                                                        prev_direction_car1[i])

        # Move car2s
        for i, car2_location in enumerate(car2_image_locations):
            car2_image_locations[i], prev_direction_car2[i] = move_car2(car2_location, rows, cols,
                                                                        house_locations, robot_location,
                                                                        ambulance_location, car1_image_locations,
                                                                        prev_direction_car2[i])

        # Draw grid with entities
        draw_grid_with_entities(screen, screen_width, screen_height - 50, rows, cols, house_image,
                                house_locations, robot_image, robot_location, ambulance_image, ambulance_location,
                                car1_image, car1_image_locations, car2_image, car2_image_locations)
        # Generate random delivery points
        if(obj1.static_variable==0):
            delivery_points = generate_delivery_points(rows, cols, house_locations + [robot_location] + [ambulance_location] + car1_image_locations + car2_image_locations)
        
        if(obj1.static_variable<=5 and check_robot==0):
            
        # Create buttons for delivery points
            delivery_buttons_rects = delivery_buttons(delivery_points, cell_width, cell_height)

        # Resize the delivery point image based on cell size
            point_image = pygame.image.load("point.jpg")
            point_image_width = cell_width  # Adjust the width to half of the cell width
            point_image_height = cell_height  # Adjust the height to half of the cell height
            point_image_resized = pygame.transform.scale(point_image, (point_image_width, point_image_height))

        # Draw delivery points
            for delivery_point in delivery_points:
                delivery_x = delivery_point[1] * cell_width + (cell_width - point_image_width) // 2  # Adjust X position to center the point image
                delivery_y = delivery_point[0] * cell_height + (cell_height - point_image_height) // 2  # Adjust Y position to center the point image

            # Check if the delivery point is currently clicked
                if (delivery_point[1] * cell_width + cell_width // 2,
                    delivery_point[0] * cell_height + cell_height // 2) in clicked_points:
                # Create a red transparent surface
                    red_surface = pygame.Surface((point_image_width, point_image_height), pygame.SRCALPHA)
                    red_surface.fill((255, 0, 0, 128))  # Red color with 50% transparency

                # Blit the red transparent surface onto the delivery point image
                    screen.blit(point_image_resized, (delivery_x, delivery_y))
                    screen.blit(red_surface, (delivery_x, delivery_y))
                else:
                # Draw the delivery point image
                    screen.blit(point_image_resized, (delivery_x, delivery_y))

        if goal_location is not None:
            optimal_path_bfs , time_com_bfs , space_bfs = best_first_search(robot_location, goal_location, house_locations)
            costs = generate_costs_15x15()
            optimal_path_a_star , time_com_a_star , space_a_star = a_star(robot_location, goal_location, house_locations,costs)
            if(time_com_a_star>time_com_bfs and space_a_star>space_bfs):
                optimal_path = optimal_path_bfs
                time_com = time_com_bfs
                space = space_bfs
                # print("best_first_search")

            else:
                optimal_path = optimal_path_a_star
                time_com = time_com_a_star
                space = space_a_star
                # print("a*")                   
            
            # print("time: " , time_com)
            # print("space: " , space)

            if optimal_path:
                # Move the robot along the optimal path
                if len(optimal_path) > 1:  # Check if there are steps to move
                    next_location = optimal_path[1]  # Get the next location to move
                    if is_occupied(next_location, car1_image_locations, car2_image_locations, ambulance_location):
                        # print("Next cell is occupied by a car. Waiting for", CAR_CHECK_DELAY, "seconds...")
                        time.sleep(CAR_CHECK_DELAY)
    
                        if not is_occupied(next_location, car1_image_locations, car2_image_locations, ambulance_location):
        # If the cell is now free, move the robot
                            robot_location = next_location
                    else:
                        robot_location = next_location
                    # print("Robot Location:", robot_location)
            # print("Robot Location:", robot_location)

        if robot_location == goal_location:
            
              # Check if the robot reaches the goal and remove the delivery point
            for i, point in enumerate(delivery_points):
                if point == goal_location:
                    del delivery_points[i]
                    if (len(item_name) == 0):

                        check_robot=1  # Remove the delivery point from the list
                    break  # Exit the loop after removing the point

        # Draw item buttons
        draw_item_buttons(screen, item_images)

        pygame.display.flip()
        pygame.time.delay(CAR_MOVEMENT_DELAY)

    pygame.quit()

if __name__ == "__main__":
    main()
